# Supernova Type 1a 
follow Chapter 4 of <a href=https://arxiv.org/pdf/1411.3596.pdf> Standardization of type Ia supernovae </a> <br>
with data from <a href=https://csp.obs.carnegiescience.edu/data/CSP_Photometry_DR2.tar.gz/view> Carnegie Supernova Project</a> <br>
additional Info <a href=http://www.physik.uni-regensburg.de/forschung/gebhardt/gebhardt_files/skripten/Entfernungsbest.pdf> Entfernungsmessung mit SNs </a>

In [ ]:
import glob
import re
import subprocess

import numpy as np
from scipy import optimize, interpolate
import matplotlib.pyplot as plt

## 1. Load Data

In [ ]:
lightcurves = {}
zcmb = {}

use_only = ["2004ef", "2004eo", "2004ey", "2005M", "2005hc", "2005iq", "2005kc", "2006X", "2006ax", "2006bh", "2007af", "2007le", "2007on", "2008bc", "2008fp", "2008gp", "2008hv"]

subprocess.run(["curl", "-o", "data.tar.gz", "https://csp.obs.carnegiescience.edu/data/CSP_Photometry_DR2.tar.gz"])
subprocess.run(["tar", "-xf", "data.tar.gz"])

for f in glob.glob('./CSP_Photometry_DR2/*.dat'):
    sn_name = re.search("SN(.*)opt", f).group(1)
    if not sn_name in use_only:
        continue
        
    lc_raw = np.loadtxt(f)
    time_raw = lc_raw.T[0]
    flux_raw = -lc_raw.T[3]

    mask = np.where(flux_raw != -99.9)
    time_filtered = time_raw[mask]
    flux_filtered = flux_raw[mask]
    
    # alternative without numpy: list comprehension
    # time_filtered = [t for t, f in zip(time_raw, flux_raw) if f != 99.990]
    # flux_filtered = [f for f in flux_raw if f != 99.990]
    
    lightcurves[sn_name] = (time_filtered, flux_filtered)
    with open(f, mode="r") as fp:
        zcmb_str = re.search("zcmb = ([0-9.]*)", fp.read()).group(1)
    
    zcmb[sn_name] = float(zcmb_str)


### 1.2. all LCs

In [ ]:
def show_lcs(lightcurves, figsize=None):
    fig, ax = plt.subplots()
    
    if not figsize is None:
        fig.set_size_inches((14, 4))
    
    for _, (lc_time, lc_flux) in lightcurves.items():
        ax.plot(lc_time, lc_flux, marker='+', linewidth=0)

show_lcs(lightcurves, figsize=(14, 4))

## 2. Time Axis Correction
define a common time scale: <br>
express time axis as phase = time - time of maxiumum 

## 3. Redshift and Distance Correction 

change from emission to observer rest frame causes

* change of timescale according to:  $\frac{\Delta t_{obs}}{\Delta t_{em}} = z+1$ (only small change since all are close)

* offset of the magnitude due to both the redshift and the distance <br>
we don't know distance and afterwards magnitues might still not coincide because of other differences in absolute magnitude <br>
so we just take mean of all peaks and move (the peak of) each SN to that value




## 4. Stretch Correction
the width difference in the light curves is associated to their intrinsic brightness (broader ↔ brighter)<br>
compensate brightness difference by stretching it